In [14]:
from os import listdir

file_path_1 = "C:/Users/Leste/OneDrive - Johns Hopkins/Desktop/BDD data/extracted/training-RiskFactors-Complete-Set1/"
file_path_2 = "C:/Users/Leste/OneDrive - Johns Hopkins/Desktop/BDD data/extracted/training-RiskFactors-Complete-Set2/"
file_path_3 = "C:/Users/Leste/OneDrive - Johns Hopkins/Desktop/BDD data/extracted/testing-RiskFactors-Complete/"
names_1 = [f for f in listdir(file_path_1) if f.endswith('.xml')]
names_2 = [f for f in listdir(file_path_2) if f.endswith('.xml')]
names_3 = [f for f in listdir(file_path_3) if f.endswith('.xml')] 

In [15]:
import pandas as pd
import numpy as np

all1_df = pd.DataFrame(np.zeros((len(names_1), 2), dtype=object), columns=['text', 'annotation'])
all2_df = pd.DataFrame(np.zeros((len(names_2), 2), dtype=object), columns=['text', 'annotation'])
test_df = pd.DataFrame(np.zeros((len(names_3), 2), dtype=object), columns=['text', 'annotation'])

In [16]:
import re
import xml.etree.ElementTree as ET


def to_df(names,df, file_path,PHI_status = True):
    #get text
    n = 0
    for name in names:
        tree = ET.parse(file_path + name)
        root = tree.getroot()

        ## Get the text
        nt = re.sub('\n',' ',root[0].text)
        nt = re.sub('\t',' ',nt) 
        nt = re.sub('"',"'",nt)
        ## sample 214 has a weird character
        nt = re.sub('>','&gt;',nt) 
        nt = re.sub('<','&lt;',nt)
        ## new wired character
        nt = re.sub('Â',' ',nt)
        nt = re.sub('â',' ',nt)
        nt = re.sub('€',' ',nt)
        nt = re.sub('™',' ',nt)
        df['text'][n] = nt
        n+=1
    
    #get annotations
    n = 0
    for name in names:
        tree = ET.parse(file_path + name)
        root = tree.getroot()
        ## Get the labels

        tag_list = []
        # get PHI labels if there are any
        if PHI_status == True:
            PHI = [root[1][x].tag for x in range(len(root[1]))].index('PHI')
            for k in range(PHI,len(root[1])):
                tag_list.append((root[1][k].attrib['text'],root[1][k].tag))
        
        # get the rest of labels
        for k in range(len(root[1])):
            for m in range(len(root[1][k])):
                if root[1][k][m].attrib.keys().__contains__('text') == False:
                    continue
                tag_list.append((root[1][k][m].attrib['text'],root[1][k][m].tag))
        
        df['annotation'][n] = tag_list
        n+=1
    return df

all_1 = to_df(names_1,all1_df, file_path_1, PHI_status = False)
all_2 = to_df(names_2,all2_df, file_path_2, PHI_status = False)
test = to_df(names_3,test_df, file_path_3, PHI_status = False)

In [17]:
train = pd.concat([all_1, all_2], ignore_index=True)

# modify testing text (change coronary arterary disease to CAD ...)

In [ ]:
test['text'] = test['text'].apply(lambda x: re.sub('coronary artery disease','CAD',x))
test['text'] = test['text'].apply(lambda x: re.sub('Blood Pressure','BP',x))
test['text'] = test['text'].apply(lambda x: re.sub('blood pressure','BP',x))
test['text'] = test['text'].apply(lambda x: re.sub('Blood pressure','BP',x))
test['text'] = test['text'].apply(lambda x: re.sub('blood Pressure','BP',x))
test['text'] = test['text'].apply(lambda x: re.sub('Blood pressure','BP',x))
test['text'] = test['text'].apply(lambda x: re.sub('&#8211','',x))
test['text'] = test['text'].apply(lambda x: re.sub('p\.o\.','per oral',x))
test['text'] = test['text'].apply(lambda x: re.sub('h/o','had',x))

## try to figure out negation
test['text'] = test['text'].apply(lambda x: re.sub("don't have ",'no_',x))

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
sw = stopwords.words('english')
for i in range(test.shape[0]):
    for x in range(len(test['annotation'][i])):
        word_tokens = word_tokenize(test['annotation'][i][x][0])
        # converts the words in word_tokens to lower case and then checks whether 
        #they are present in stop_words or not
        filtered_sentence = [w for w in word_tokens if not w.lower() in sw]
        tagged_things = ' '.join(filtered_sentence)
        tagged_things = re.sub('coronary artery disease','CAD',tagged_things)
        tagged_things = re.sub('Blood Pressure','BP',tagged_things)
        tagged_things = re.sub('blood pressure','BP',tagged_things)
        tagged_things = re.sub('Blood pressure','BP',tagged_things)
        tagged_things = re.sub('blood Pressure','BP',tagged_things)
        tagged_things = re.sub('&#8211','',tagged_things)
        tagged_things = re.sub('p\.o\.','per oral',tagged_things)
        tagged_things = re.sub('h/o','had',tagged_things)
        tagged_things = re.sub("don't have ",'no_',tagged_things)
        test['annotation'][i][x] = (tagged_things,test['annotation'][i][x][1])

In [ ]:
train['text'] = train['text'].apply(lambda x: re.sub('coronary artery disease','CAD',x))
train['text'] = train['text'].apply(lambda x: re.sub('Blood Pressure','BP',x))
train['text'] = train['text'].apply(lambda x: re.sub('blood pressure','BP',x))
train['text'] = train['text'].apply(lambda x: re.sub('Blood pressure','BP',x))
train['text'] = train['text'].apply(lambda x: re.sub('blood Pressure','BP',x))
train['text'] = train['text'].apply(lambda x: re.sub('Blood pressure','BP',x))
train['text'] = train['text'].apply(lambda x: re.sub('&#8211','',x))
train['text'] = train['text'].apply(lambda x: re.sub('p\.o\.','per oral',x))
train['text'] = train['text'].apply(lambda x: re.sub('h/o','had',x))

## try to figure out negation
train['text'] = train['text'].apply(lambda x: re.sub("don't have ",'no_',x))


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
sw = stopwords.words('english')
for i in range(train.shape[0]):
    for x in range(len(train['annotation'][i])):
        word_tokens = word_tokenize(train['annotation'][i][x][0])
        # converts the words in word_tokens to lower case and then checks whether 
        #they are present in stop_words or not
        filtered_sentence = [w for w in word_tokens if not w.lower() in sw]
        tagged_things = ' '.join(filtered_sentence)
        tagged_things = re.sub('coronary artery disease','CAD',tagged_things)
        tagged_things = re.sub('Blood Pressure','BP',tagged_things)
        tagged_things = re.sub('blood pressure','BP',tagged_things)
        tagged_things = re.sub('Blood pressure','BP',tagged_things)
        tagged_things = re.sub('blood Pressure','BP',tagged_things)
        tagged_things = re.sub('&#8211','',tagged_things)
        tagged_things = re.sub('p\.o\.','per oral',tagged_things)
        tagged_things = re.sub('h/o','had',tagged_things)
        tagged_things = re.sub("don't have ",'no_',tagged_things)
        train['annotation'][i][x] = (tagged_things,train['annotation'][i][x][1])


In [ ]:

for i in range(train.shape[0]):
    word_tokens = word_tokenize(train['text'][i])
    filtered_sentence = [w for w in word_tokens if not w.lower() in sw]
    train['text'][i] = ' '.join(filtered_sentence)

for i in range(test.shape[0]):
    word_tokens = word_tokenize(test['text'][i])
    filtered_sentence = [w for w in word_tokens if not w.lower() in sw]
    test['text'][i] = ' '.join(filtered_sentence)


In [ ]:
from tqdm import tqdm
from difflib import SequenceMatcher
import pickle

def matcher(string, pattern):
    '''
    Return the start and end index of any pattern present in the text.
    '''
    match_list = []
    pattern = pattern.strip()
    seqMatch = SequenceMatcher(None, string, pattern, autojunk=False)
    match = seqMatch.find_longest_match(0, len(string), 0, len(pattern))
    if (match.size == len(pattern)):
        start = match.a
        end = match.a + match.size
        match_tup = (start, end)
        string = string.replace(pattern, "X" * len(pattern), 1)
        match_list.append(match_tup)
        
    return match_list, string


#def mark_sentence(s, match_list):
#    '''
#    Marks all the entities in the sentence as per the BIO scheme. 
#    '''
#    word_dict = {}
#    for word in s.split():
#        word_dict[word] = 'O'
#        
#    for start, end, e_type in match_list:
#        temp_str = s[start:end]
#        tmp_list = temp_str.split()
#        if len(tmp_list) > 1:
#            word_dict[tmp_list[0]] = 'B-' + e_type
#            for w in tmp_list[1:]:
#                word_dict[w] = 'I-' + e_type
#        else:
#            word_dict[temp_str] = 'B-' + e_type
#    return word_dict

## replace !!mark_sentence!! to better label the text with more than one word
""" def create_labs(s,match_list):
    labs = ['O' for i in range(len(s.split()))]
    word_dict = pd.DataFrame({'word':s.split(),'label':labs})

    for start, end, e_type in match_list:
        index = len(re.findall(r' +',s[0:start]))-1
        num_words = len(s[start:end].split())

        if num_words > 1:
            word_dict.loc[index,'label'] = 'B-' + e_type
            for i in range(1,num_words):
                word_dict.loc[index+i,'label'] = 'I-' + e_type
        else:
            word_dict.loc[index,'label'] = 'B-' + e_type
    return word_dict """

def create_labs(s,match_list):
    labs = ['O' for i in range(len(s.split()))]
    word_dict = pd.DataFrame({'word':s.split(),'label':labs})

    for start, end, e_type in match_list:
        index = len(re.findall(r' +',s[0:start]))-1
        num_words = len(s[start:end].split())

        if num_words > 1:
            word_dict.loc[index,'label'] = e_type
            for i in range(1,num_words):
                word_dict.loc[index+i,'label'] = e_type
        else:
            word_dict.loc[index,'label'] = e_type
    return word_dict

def clean(text):
    '''
    Just a helper fuction to add a space before the punctuations for better tokenization
    '''
    filters = ["!", "#", "$", "%", "&", "(", ")", "/", "*", ".", ":", ";", "<", "=", ">", "?", "@", "[",
               "\\", "]", "_", "`", "{", "}", "~", "'"]
    for i in text:
        if i in filters:
            text = text.replace(i, " " + i)
            
    return text

def to_txt(df, filepath):
    '''
    The function responsible for the creation of data in the said format.
    '''
    with open(filepath , 'w') as f:
        for text, annotation in zip(df.text, df.annotation):
            #text = clean(text)
            #text_ = text    
            print(df.index[df['text']== text].tolist())    
            match_list = []
            for i in annotation:
                a, text_ = matcher(text, i[0])
                match_list.append((a[0][0], a[0][1], i[1]))

            d = create_labs(text, match_list)

            #for i in d.keys():
            #    f.writelines(i + ' ' + d[i] +'\n')
            #f.writelines('\n')
            for i in range(d.shape[0]):
                f.writelines(d['word'][i] + ' ' + d['label'][i] +'\n')
            f.writelines('\n')
def to_csv(df, filepath):
    for text, annotation in zip(df.text, df.annotation):
            #text = clean(text)
            #text_ = text    
            print(df.index[df['text']== text].tolist())    
            match_list = []
            for i in annotation:
                a, text_ = matcher(text, i[0])
                match_list.append((a[0][0], a[0][1], i[1]))

            d = create_labs(text, match_list)
            d.to_csv(filepath, index = False, header = False, mode = 'a')
            
def main(input,save_path):
    ## An example dataframe.
    #data = pd.DataFrame([["Horses are too tall and they pretend to care about your feelings", [("Horses", "ANIMAL")]],
    #              ["Who is Shaka Khan?", [("Shaka Khan", "PERSON")]],
    #              ["I like London and Berlin.", [("London", "LOCATION"), ("Berlin", "LOCATION")]],
    #              ["There is a banyan tree in the courtyard", [("banyan tree", "TREE")]]], columns=['text', 'annotation'])
    data = input
    ## path to save the txt file.
    ## creating the file.
    to_txt(data, save_path)
    
#if __name__ == '__main__':
path = 'C:/Users/Leste/OneDrive - Johns Hopkins/Documents/GitHub/nlpsumm/BERT/data_processed/'

#main(train,path+'train.txt')
main(train,path+'train_clean(no_BIO).txt')
main(test,path+'test_clean(no_BIO).txt')

In [ ]:
train

In [ ]:
# checking p.o. in training labels
for i in range(test.shape[0]):
    for x in range(len(train['annotation'][i])):
        y=re.search("&#8211",train['annotation'][i][x][0])
        if y:
            print(train['annotation'][i][x][0])
            print(i)


In [18]:

for i in range(test.shape[0]):
    x = re.search("&#8211",test['text'][i])
    if x:
        print(x)
        print(i)

<re.Match object; span=(2856, 2862), match='&#8211'>
21
<re.Match object; span=(2582, 2588), match='&#8211'>
61
<re.Match object; span=(2670, 2676), match='&#8211'>
71
<re.Match object; span=(4276, 4282), match='&#8211'>
181
<re.Match object; span=(5780, 5786), match='&#8211'>
183
<re.Match object; span=(1400, 1406), match='&#8211'>
218
<re.Match object; span=(66, 72), match='&#8211'>
268
<re.Match object; span=(62, 68), match='&#8211'>
269
<re.Match object; span=(1651, 1657), match='&#8211'>
270
<re.Match object; span=(3049, 3055), match='&#8211'>
311
<re.Match object; span=(1344, 1350), match='&#8211'>
374
<re.Match object; span=(1163, 1169), match='&#8211'>
392
<re.Match object; span=(5701, 5707), match='&#8211'>
393
<re.Match object; span=(2349, 2355), match='&#8211'>
439
<re.Match object; span=(1035, 1041), match='&#8211'>
476
<re.Match object; span=(56, 62), match='&#8211'>
502
<re.Match object; span=(850, 856), match='&#8211'>
503
<re.Match object; span=(1116, 1122), match='&#82

In [19]:
names_3[513]

'389-04.xml'